# GPT-SoVITS on Sagemaker

## build image

In [48]:
!chmod +x ./*.sh && ./build_and_push.sh 

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Cloning into 'GPT-SoVITS'...
remote: Enumerating objects: 2638, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 2638 (delta 47), reused 70 (delta 41), pack-reused 2562
Receiving objects: 100% (2638/2638), 6.47 MiB | 23.09 MiB/s, done.
Resolving deltas: 100% (1449/1449), done.
Sending build context to Docker daemon  26.18MB
Step 1/22 : FROM cnstark/pytorch:2.0.1-py3.9.17-cuda11.8.0-ubuntu20.04
 ---> 8fd9e4c5e7bc
Step 2/22 : ARG IMAGE_TYPE=sagemaker-byoc
 ---> Using cache
 ---> 61c671c10000
Step 3/22 : RUN mkdir -p /opt/program
 ---> Using cache
 ---> 55ab582b8ed0
Step 4/22 : RUN chmod 777 /opt/program
 ---> Usin

In [49]:
import boto3
import sagemaker
from sagemaker import Model, image_uris, serializers, deserializers

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment
bucket = sess.default_bucket()
image="gpt-sovits-inference"
s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

full_image_uri=f"{account_id}.dkr.ecr.{region}.amazonaws.com/{image}:latest"
print(full_image_uri)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
687912291502.dkr.ecr.us-west-2.amazonaws.com/gpt-sovits-inference:latest


## remote debug test

In [50]:
## empty model data for byoc with webserver
!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'gpt_sovits')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'gpt_sovits')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

dummy
upload: ./model.tar.gz to s3://sagemaker-us-west-2-687912291502/gpt_sovits/assets/model.tar.gz


In [51]:
from sagemaker_ssh_helper.wrapper import SSHModelWrapper
model = Model(image_uri=full_image_uri, model_data=model_data, role=role,dependencies=[SSHModelWrapper.dependency_dir()] )

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [52]:
from sagemaker_ssh_helper.wrapper import SSHModelWrapper
instance_type = "ml.g5.xlarge"
endpoint_name = sagemaker.utils.name_from_base("gpt-sovits-inference")


ssh_wrapper = SSHModelWrapper.create(model, connection_wait_time_seconds=0)  # <--NEW--

predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    wait=False
)


#instance_ids = ssh_wrapper.get_instance_ids(timeout_in_sec=900)  # <--NEW-- 
#print(f"To connect over SSM run: aws ssm start-session --target {instance_ids[0]}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [53]:
instance_ids = ssh_wrapper.get_instance_ids(timeout_in_sec=0)

In [ ]:
instance_ids[0]

## SM endpoint test

### create sagemaker model

In [ ]:
import boto3
import re
import os
import json
import uuid
import boto3
import sagemaker
from time import gmtime, strftime
## for debug only
from sagemaker_ssh_helper.wrapper import SSHModelWrapper
sm_client = boto3.client(service_name='sagemaker')



def create_model():
    image=full_image_uri
    model_name="gpt-sovits-sagemaker-"+strftime("%Y-%m-%d-%H-%M-%S", gmtime())
    create_model_response = sm_client.create_model(
        ModelName=model_name,
        ExecutionRoleArn=role,
        Containers=[{"Image": image}],
    )
    print(create_model_response)
    return model_name

In [ ]:
model_name=create_model()


### create endpoint configuration

In [ ]:
endpointConfigName = "gpt-sovits-sagemaker-configuration-"+strftime("%Y-%m-%d-%H-%M-%S", gmtime())
def create_endpoint_configuration():
    create_endpoint_config_response = sm_client.create_endpoint_config(     
        EndpointConfigName=endpointConfigName,
        ProductionVariants=[
            {
                #"ModelName":"gpt-sovits-sagemaker-012024-03-28-04-00-03",
                "ModelName":model_name,
                "VariantName": "gpt-sovits-sagemaker"+"-variant",
                "InstanceType": "ml.g5.xlarge",  # 指定 g5.xlarge 机器
                "InitialInstanceCount": 1,
                "ModelDataDownloadTimeoutInSeconds": 1200,
                "ContainerStartupHealthCheckTimeoutInSeconds": 1200
            }
        ],
    )
    print(create_endpoint_config_response)
    return endpointConfigName


In [ ]:
create_endpoint_configuration()


### create endpoint

In [ ]:
endpointName="gpt-sovits-sagemaker-endpoint"+strftime("%Y-%m-%d-%H-%M-%S", gmtime())
def create_endpoint():
    create_endpoint_response = sm_client.create_endpoint(
        EndpointName=endpointName,
        #EndpointConfigName="gpt-sovits-sagemaker-configuration2024-03-28-04-03-53",
        EndpointConfigName=endpointConfigName
    )
    print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])
    resp = sm_client.describe_endpoint(EndpointName=endpointName)
    print("Endpoint Status: " + resp["EndpointStatus"])
    print("Waiting for {} endpoint to be in service".format("gpt-sovits-sagemaker-endpoint"))
    waiter = sm_client.get_waiter("endpoint_in_service")
    waiter.wait(EndpointName=endpointName)

In [ ]:
create_endpoint()

## Realtime inferecne with sagemaker endpoint

In [ ]:
import json
import boto3
endpointName="gpt-sovits-inference-2024-05-07-08-46-43-537"
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")
#endpointName="gpt-sovits-sagemaker-endpoint2024-04-03-23-49-44"


request = {"refer_wav_path":"s3://sagemaker-us-west-2-687912291502/gpt-sovits/wav/speech_20240425104005663.mp3",
    "prompt_text": "私はスポーツが好きな女の子で、私は中華料理が大好きで、私は中国へ旅行するのが好きで、特に杭州、成都が好きです",
    "prompt_language":"ja",
    "text":"あなたは四海を家とすることを約束します,私を待っていても気にしないで、あなたの白髪を許す",
    "text_language" :"ja",
    "output_s3uri":"s3://sagemaker-us-west-2-687912291502/gpt_sovits_output/wav/"}


request = {"refer_wav_path":"s3://sagemaker-us-west-2-687912291502/gpt-sovits/wav/123.WAV.wav",
    "prompt_text": "早上好，欢迎来到我的一天，这是我做音频主播的第四个年头了，有小伙伴留言想让我分享一些音频直播的经验，今天我先和大家聊聊我的入行原因。其实我从中选情就很喜欢电台了，在校期间眼睛记得参加各类广播站以及校园主持的活动，我好像对话筒和声音就有一种莫名的执念，所以在17年里的时候，我创建了一个自己的电台公众号，通过声音和文字记录自己的一些心事。后来18年底经由朋友的介绍，可以通过音频直播分享我写的东西，还有我的声音。当时呢我就想音频直播又不用落脸那么方便，试一试吧，如果有人喜欢，还可以给自己的电台公众号吸吸粉。后来我就在直播间里认识了越来越多的听友，渐渐的这份工作，也为我带来了一些兼职收入，我就决定把这份工作做下去。",
    "prompt_language":"zh",
    "text":"作为SAP基础架构专家,我来解释一下SAP Basis的含义:SAP Basis是指SAP系统的基础设施层,负责管理和维护整个SAP系统环境的运行。它包括以下几个主要方面:SAP系统管理包括SAP系统实例的安装、启动、监控、备份、升级等日常管理任务。Basis团队负责保证系统的正常运行。",
    "text_language" :"zh",
    "output_s3uri":"s3://sagemaker-us-west-2-687912291502/gpt_sovits_output/wav/"}






def invoke_endpoint():
    content_type = "application/json"
    request_body = request
    payload = json.dumps(request_body)
    print(payload)
    response = runtime_sm_client.invoke_endpoint(
        EndpointName=endpointName,
        ContentType=content_type,
        Body=payload,
    )
    result = response['Body'].read().decode()
    print('返回：',result)

In [ ]:
response=invoke_endpoint()

In [ ]:
!aws s3 cp s3://sagemaker-us-west-2-687912291502/gpt-sovits/wav/speech_20240425104005663.mp3 ./
!aws s3 cp s3://sagemaker-us-west-2-687912291502/gpt_sovits_output/wav/gpt_sovits_1715140344.wav ./

In [ ]:
!aws s3 cp s3://sagemaker-us-west-2-687912291502/gpt_sovits_output/wav/gpt_sovits_1715150796.wav ./

## Streams test (only for stream branch deployment)

In [64]:
import requests

chunk_bytes=None

def upsert(lst, new_dict):
    for i, item in enumerate(lst):
        if new_dict['index'] == i:
            lst[i] = new_dict
            return lst
    lst.append(new_dict)
    return lst

def invoke_streams_endpoint(smr_client,endpointName, request):
    global chunk_bytes
    content_type = "application/json"
    payload = json.dumps(request,ensure_ascii=False)

    response_model = smr_client.invoke_endpoint_with_response_stream(
        EndpointName=endpointName,
        ContentType=content_type,
        Body=payload,
    )

    result = []
    print(response_model['ResponseMetadata'])
    event_stream = iter(response_model['Body'])
    index = 0
    try: 
        while True:
            event = next(event_stream)
            eventChunk = event['PayloadPart']['Bytes']
            chunk_dict = {}
            if index == 0:
                print("Received first chunk")
                chunk_dict['first_chunk'] = True
                chunk_dict['bytes'] = eventChunk
                chunk_bytes = eventChunk
                chunk_dict['last_chunk'] = False
                chunk_dict['index'] = index
            else:
                chunk_dict['first_chunk'] = False
                chunk_dict['bytes'] = eventChunk
                chunk_bytes = eventChunk
                chunk_dict['last_chunk'] = False
                chunk_dict['index'] = index
            print("chunk len:",len(chunk_dict['bytes']))
            result.append(chunk_dict)    
            index += 1
            #print('返回chunk：', chunk_dict['bytes'])
    except StopIteration:
        print("All chunks processed")
        chunk_dict = {}
        chunk_dict['first_chunk'] = False
        chunk_dict['bytes'] = chunk_bytes
        chunk_dict['last_chunk'] = True
        chunk_dict['index'] = index-1
        result = upsert(result,chunk_dict)
    print("result",result)
    return result





In [65]:
import json
import boto3
endpointName="gpt-sovits-inference-2024-05-17-13-49-58-483"
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")
#endpointName="gpt-sovits-sagemaker-endpoint2024-04-03-23-49-44"



request = {"refer_wav_path":"s3://sagemaker-us-west-2-687912291502/gpt-sovits/wav/speech_20240425104005663.mp3",
    "prompt_text": "私はスポーツが好きな女の子で、私は中華料理が大好きで、私は中国へ旅行するのが好きで、特に杭州、成都が好きです",
    "prompt_language":"ja",
    "text":"『白夜行』はとても美しい小説で、私はとても夢中になって読んで、時には何時間も休まないで、私は中の主人公が大好きです",
    "text_language" :"ja",
    "output_s3uri":"s3://sagemaker-us-west-2-687912291502/gpt_sovits_output/wav/",
    "cut_punc":"、"}


In [66]:
response=invoke_streams_endpoint(runtime_sm_client,endpointName,request)

{'RequestId': '46027a69-b658-4d8a-a783-b1b02e9fa34d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '46027a69-b658-4d8a-a783-b1b02e9fa34d', 'x-amzn-invoked-production-variant': 'AllTraffic', 'x-amzn-sagemaker-content-type': 'application/json', 'date': 'Fri, 17 May 2024 14:20:27 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive'}, 'RetryAttempts': 0}
Received first chunk
chunk len: 95
chunk len: 95
chunk len: 95
chunk len: 95
All chunks processed
result [{'first_chunk': True, 'bytes': b'{"result": "s3://sagemaker-us-west-2-687912291502/gpt_sovits_output/wav/gpt_sovits_1715955628"}', 'last_chunk': False, 'index': 0}, {'first_chunk': False, 'bytes': b'{"result": "s3://sagemaker-us-west-2-687912291502/gpt_sovits_output/wav/gpt_sovits_1715955629"}', 'last_chunk': False, 'index': 1}, {'first_chunk': False, 'bytes': b'{"result": "s3://sagemaker-us-west-2-687912291502/gpt_sovits_output/wav/gpt_sovits_1715955631"}'

In [60]:
!aws s3 cp s3://sagemaker-us-west-2-687912291502/gpt_sovits_output/wav/gpt_sovits_1715954949 ./

download: s3://sagemaker-us-west-2-687912291502/gpt_sovits_output/wav/gpt_sovits_1715954949 to ./gpt_sovits_1715954949
